In [135]:
import pandas as pd

## Retail DF (After feb)

In [136]:
clean = pd.read_csv('/home/arizona/code/ESCP/hackathon/hackathon_mbd/data/clean_hills_data_without_unknown.csv')

In [137]:
order_customer_list = set(clean[clean['Order']>0]['customer_id'].to_list())
no_order_customer_list = set(clean[clean['Order']==0]['customer_id'].to_list())

In [138]:
retail = pd.read_csv('/home/arizona/code/ESCP/hackathon/hackathon_mbd/data/retailer.csv')

In [139]:
retail_order = retail[retail['customer_id'].isin(order_customer_list)]
retail_no_order = retail[retail['customer_id'].isin(no_order_customer_list)]

In [140]:
customer_order = retail_order[retail_order['quantity']>0]

In [141]:
customer_order['timestamp_utc'] = pd.to_datetime(customer_order['timestamp_utc'])
retail_no_order['timestamp_utc'] = pd.to_datetime(retail_no_order['timestamp_utc'])

/tmp/ipykernel_70364/661538655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_order['timestamp_utc'] = pd.to_datetime(customer_order['timestamp_utc'])
/tmp/ipykernel_70364/661538655.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retail_no_order['timestamp_utc'] = pd.to_datetime(retail_no_order['timestamp_utc'])


In [142]:
customer_order = customer_order[customer_order['customer_id']!='unknown']
retail_no_order = retail_no_order[retail_no_order['customer_id']!='unknown']

In [143]:
cust_after_feb = customer_order[customer_order['timestamp_utc']>'2024-02-29 00:00:00']
no_cust_after_feb = retail_no_order[retail_no_order['timestamp_utc']>'2024-02-29 00:00:00']

In [144]:
cust_after_feb['source_data'] = 'Website_Order'
no_cust_after_feb['source_data'] = 'Website_Order'

/tmp/ipykernel_70364/1980338913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cust_after_feb['source_data'] = 'Website_Order'
/tmp/ipykernel_70364/1980338913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_cust_after_feb['source_data'] = 'Website_Order'


In [145]:
cust_list = set(cust_after_feb['customer_id'].to_list())
no_cust_list = set(no_cust_after_feb['customer_id'].to_list())

In [146]:
len(set(cust_after_feb['customer_id'].to_list()))

396703

In [147]:
len(set(no_cust_after_feb['customer_id'].to_list()))

533489

In [148]:
# Sort data by customer and timestamp
cust_after_feb = cust_after_feb.sort_values(by=['customer_id', 'timestamp_utc'])

# Calculate time since last order for each customer
cust_after_feb['time_since_last_order'] = cust_after_feb.groupby('customer_id')['timestamp_utc'].diff()

# Calculate the average time between orders per customer
order_frequency = cust_after_feb.groupby('customer_id')['time_since_last_order'].mean().reset_index()
order_frequency.columns = ['customer_id', 'average_time_between_orders']

# Merge back to the original DataFrame so each row has the avg order frequency of that customer
cust_after_feb = cust_after_feb.merge(order_frequency, on='customer_id', how='left')

# Display results
cust_after_feb.head()

,customer_id,timestamp_utc,event_name,brand,product_name,sales,quantity,source_data,time_since_last_order,average_time_between_orders
0,re000fIO9QXTWYjOfn,2024-03-13 02:58:00,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,Website_Order,NaT,33 days 14:22:13
1,re000fIO9QXTWYjOfn,2024-05-24 18:10:37,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,Website_Order,72 days 15:12:37,33 days 14:22:13
2,re000fIO9QXTWYjOfn,2024-06-04 18:27:57,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,Website_Order,11 days 00:17:20,33 days 14:22:13
3,re000fIO9QXTWYjOfn,2024-06-21 22:04:39,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,Website_Order,17 days 03:36:42,33 days 14:22:13
4,re000pHbVOysCXRHgt,2024-03-27 16:13:49,Order,Science Diet,SD Ca Adt Lt Ckn 30lb bg,78.99,1.0,Website_Order,NaT,36 days 10:14:38.500000


In [149]:
cust_after_feb['average_time_between_orders'].mean()

Timedelta('25 days 15:13:26.496426060')

In [150]:
cust_after_feb[cust_after_feb['customer_id']=='re000pHbVOysCXRHgt']

,customer_id,timestamp_utc,event_name,brand,product_name,sales,quantity,source_data,time_since_last_order,average_time_between_orders
4,re000pHbVOysCXRHgt,2024-03-27 16:13:49,Order,Science Diet,SD Ca Adt Lt Ckn 30lb bg,78.99,1.0,Website_Order,NaT,36 days 10:14:38.500000
5,re000pHbVOysCXRHgt,2024-04-30 14:17:03,Order,Science Diet,SD Ca Adt Lt Ckn 30lb bg,78.99,1.0,Website_Order,33 days 22:03:14,36 days 10:14:38.500000
6,re000pHbVOysCXRHgt,2024-06-08 12:43:06,Order,Science Diet,SD Ca Adt Lt Ckn 30lb bg,78.98,1.0,Website_Order,38 days 22:26:03,36 days 10:14:38.500000


## Map with Programmatic df

In [151]:
map = pd.read_csv('/home/arizona/code/ESCP/hackathon/hackathon_mbd/data/mapping_transac_publisher_tv.csv')

In [152]:
map_cust = map[map['customer_id'].isin(cust_list)]
map_cust = map_cust[map_cust['dsp_id']!= 'unknown']

no_map_cust = map[map['customer_id'].isin(no_cust_list)]
no_map_cust = no_map_cust[no_map_cust['dsp_id']!= 'unknown']

In [153]:
map_cust

,customer_id,dsp_id,device_id
2,reOrpt9vhSwhbPVtni,dsp1hXcI9Q6TZYzLEmeTkxzhjqD6HJ,ctvwp5n34myx
3,reutQ3jiBX9Li4Ggqi,dspcd3UcXUcUk0PEo2hb8CEH3WVlFE,ctvdkYC70D2x
11,regQRb3YkfbTywAYqb,dsp5KAMATcrdRJ6xyE7rH5IoWSeRlT,ctvq09wKpvHo
12,recJAdgCooFk0ELo69,dspHecQXxyZfxt1dB0evvMjxXZFaCR,ctvgCOlTgYBq
14,re6T42jcETmJKTgh9D,dspPzJqYcwfR10Bo0i0YhJzQQLcGS5,ctvVMDmPSumY
...,...,...,...
1381730,reCBJYWc1iGg7FZzCL,dspxaquDLzn10gBI4ro57sqHTs7DBb,ctvaJGiaXsf4
1381733,reypdmZz1CIhFsWSf7,dspuOSizhUWHSnEy32ty07CyvbuQeG,ctvxyufprBYF
1381734,relyf2wXitGcdWECun,dspqsnKF0smUamj3QeCWFDs1kio4T8,ctv4uV1UmyM1
1381736,reepFoP0vHcREJs9sQ,dspsrnEAI0NywPxrkVvyUGUSvv1kNb,ctvl3kAUaLNJ


In [154]:
no_map_cust

,customer_id,dsp_id,device_id
0,reFs5GI87lXJkJSi9r,dsp9tnGII5BeXbn6LUSFZPcKGCyI0F,ctv81YlbBXho
4,reH7UgH29AreRh8wWy,dspSnORtuQRLSkZKp9nbSIpbJBQLP1,ctvTgbqnhRd8
5,reHAnmLXgIZqJT0i64,dspfp8NFN8MfPTM5zvmeSJYqX9NzSW,ctvWjRSFfqf9
7,reIYreUnTRc6yFflWy,dsp5g1hhUvLR4MjLQuI0UTQXbmQx4R,ctv5yal7dewi
9,reEW1KfvG7XlTbXwa6,dsp209Hn140h3HQO3sDplynIC1HfS4,ctvywJV5bzru
...,...,...,...
1382055,reBda2QsASNpnCCjNY,dspfCvDjSBKzqT6cD0MSiE1OvjHEHF,ctv9ZST2nvjP
1382060,reXX6EMmmiWMH2fLaS,dspoCSI8rPM1lveZ8vkcjUuPaXOYU2,ctv8Mml39De0
1382061,reLqKn5r9Mb9uDCFrV,dspQ4DIgOIwULRl3oqGVJLoyfk3ymF,ctvvvXZLmtSB
1382063,reyDBmXbnOf7TcT6by,dspSYNIBio8OfxttchfQs2gogqBSGH,ctvwDgCaXiG6


In [155]:
programmatic = pd.read_csv('/home/arizona/code/ESCP/hackathon/hackathon_mbd/data/programmatic_publisher.csv')

In [156]:
programmatic['timestamp_utc'] = pd.to_datetime(programmatic['timestamp_utc'])

In [157]:
after_feb = programmatic[programmatic['timestamp_utc']>'2024-02-29 00:00:00']

In [158]:
after_feb['cost_milli_cent'].sum()/100000

np.float64(56114.890044039916)

In [159]:
program = map_cust.merge(after_feb, how='left', on='dsp_id')
no_program = no_map_cust.merge(after_feb, how='left', on='dsp_id')

In [160]:
program = program.drop(columns=['dsp_id', 'device_id'])
no_program = no_program.drop(columns=['dsp_id', 'device_id'])

In [161]:
program = program.dropna(subset=['timestamp_utc'])
no_program = no_program.dropna(subset=['timestamp_utc'])

In [162]:
program['source_data'] = 'Programmatic_Ads'
no_program['source_data'] = 'Programmatic_Ads'

In [164]:
# Sort data by customer and timestamp
program = program.sort_values(by=['customer_id', 'timestamp_utc'])

# Calculate time since last order for each customer
program['time_since_last_ads'] = program.groupby('customer_id')['timestamp_utc'].diff()

# Calculate the average time between orders per customer
order_frequency = program.groupby('customer_id')['time_since_last_ads'].mean().reset_index()
order_frequency.columns = ['customer_id', 'average_time_between_ads_programatic']

# Merge back to the original DataFrame so each row has the avg order frequency of that customer
program = program.merge(order_frequency, on='customer_id', how='left')

# Display results
program.head()

,customer_id,timestamp_utc,campaign_name,device_type,cost_milli_cent,source_data,time_since_last_ads,average_time_between_ads_programatic
0,re002l1aOJG3OoCTK8,2024-04-08 22:13:09,Retargeting,PC,2144.539,Programmatic_Ads,NaT,NaT
1,re003crViui7MmOapY,2024-02-29 18:18:55,Retargeting,PC,3062.430,Programmatic_Ads,NaT,NaT
2,re006rhNmGIq3ZMqWJ,2024-03-03 02:14:45,Retargeting,PC,344.700,Programmatic_Ads,NaT,2 days 02:15:55.600000
3,re006rhNmGIq3ZMqWJ,2024-03-03 02:29:56,Retargeting,PC,344.700,Programmatic_Ads,0 days 00:15:11,2 days 02:15:55.600000
4,re006rhNmGIq3ZMqWJ,2024-03-08 21:01:18,Retargeting,Phone,786.321,Programmatic_Ads,5 days 18:31:22,2 days 02:15:55.600000


In [166]:
program['time_since_last_ads'].mean()

Timedelta('6 days 13:34:21.399550245')

In [ ]:
program['cost_milli_cent'].sum()/100000

np.float64(3036.3812435399996)

In [ ]:
no_program['cost_milli_cent'].sum()/100000

np.float64(11953.376453640003)

## TV

In [ ]:
map = pd.read_csv('/home/arizona/code/ESCP/hackathon/hackathon_mbd/data/mapping_transac_publisher_tv.csv')

In [ ]:
map_cust = map[map['customer_id'].isin(cust_list)]
map_cust = map_cust[map_cust['device_id']!= 'unknown']

no_map_cust = map[map['customer_id'].isin(no_cust_list)]
no_map_cust = no_map_cust[no_map_cust['device_id']!= 'unknown']

In [ ]:
map_cust

,customer_id,dsp_id,device_id
2,reOrpt9vhSwhbPVtni,dsp1hXcI9Q6TZYzLEmeTkxzhjqD6HJ,ctvwp5n34myx
3,reutQ3jiBX9Li4Ggqi,dspcd3UcXUcUk0PEo2hb8CEH3WVlFE,ctvdkYC70D2x
8,reDzpCoOpWJc5l9K3y,unknown,ctv9sCxrEjFD
11,regQRb3YkfbTywAYqb,dsp5KAMATcrdRJ6xyE7rH5IoWSeRlT,ctvq09wKpvHo
12,recJAdgCooFk0ELo69,dspHecQXxyZfxt1dB0evvMjxXZFaCR,ctvgCOlTgYBq
...,...,...,...
1381732,reHDQGzE6g88w7z7LW,unknown,ctvDkDJU6ucM
1381733,reypdmZz1CIhFsWSf7,dspuOSizhUWHSnEy32ty07CyvbuQeG,ctvxyufprBYF
1381734,relyf2wXitGcdWECun,dspqsnKF0smUamj3QeCWFDs1kio4T8,ctv4uV1UmyM1
1381736,reepFoP0vHcREJs9sQ,dspsrnEAI0NywPxrkVvyUGUSvv1kNb,ctvl3kAUaLNJ


In [ ]:
tv = pd.read_csv('/home/arizona/code/ESCP/hackathon/hackathon_mbd/data/tv_publisher.csv')

In [ ]:
tv['timestamp_utc'] = pd.to_datetime(tv['timestamp_utc'])

In [ ]:
tv_after_feb = tv[tv['timestamp_utc']>'2024-02-29 00:00:00']

In [ ]:
tv_after_feb['cost_milli_cent'].sum()/100000

np.float64(135510.60556830047)

In [ ]:
tv_map = map_cust.merge(tv_after_feb, how='left', on='device_id')
notv_map = no_map_cust.merge(tv_after_feb, how='left', on='device_id')

In [ ]:
tv_map = tv_map.dropna(subset=['timestamp_utc'])
notv_map = notv_map.dropna(subset=['timestamp_utc'])

In [ ]:
tv_map = tv_map.drop(columns=['device_id', 'dsp_id'])
notv_map = notv_map.drop(columns=['device_id', 'dsp_id'])

In [ ]:
tv_map = tv_map.rename(columns={'cost_milli_cent':'cost_milli_cent_tv'})
notv_map = notv_map.rename(columns={'cost_milli_cent':'cost_milli_cent_tv'})

In [ ]:
tv_map['source_data'] = 'TV_ads'
notv_map['source_data'] = 'TV_ads'

In [ ]:
# Sort data by customer and timestamp
tv_map = tv_map.sort_values(by=['customer_id', 'timestamp_utc'])

# Calculate time since last order for each customer
tv_map['time_since_last_ads_tv'] = tv_map.groupby('customer_id')['timestamp_utc'].diff()

# Calculate the average time between orders per customer
order_frequency = tv_map.groupby('customer_id')['time_since_last_ads_tv'].mean().reset_index()
order_frequency.columns = ['customer_id', 'average_time_between_ads_tv']

# Merge back to the original DataFrame so each row has the avg order frequency of that customer
tv_map = tv_map.merge(order_frequency, on='customer_id', how='left')

# Display results
tv_map.head()

,customer_id,timestamp_utc,cost_milli_cent_tv,source_data,time_since_last_ads_tv,average_time_between_ads_tv
0,re006iKcKMqa0lm302,2024-03-15 01:39:37,2325.51,TV_ads,NaT,74 days 04:48:10
1,re006iKcKMqa0lm302,2024-05-28 06:27:47,2325.51,TV_ads,74 days 04:48:10,74 days 04:48:10
2,re008tXVWOF6Wfii1g,2024-04-15 22:10:21,2325.51,TV_ads,NaT,7 days 21:43:45.200000
3,re008tXVWOF6Wfii1g,2024-04-17 23:50:19,2325.51,TV_ads,2 days 01:39:58,7 days 21:43:45.200000
4,re008tXVWOF6Wfii1g,2024-04-19 22:26:03,2325.51,TV_ads,1 days 22:35:44,7 days 21:43:45.200000


In [ ]:
# Sort data by customer and timestamp
notv_map = notv_map.sort_values(by=['customer_id', 'timestamp_utc'])

# Calculate time since last order for each customer
notv_map['time_since_last_ads_tv'] = notv_map.groupby('customer_id')['timestamp_utc'].diff()

# Calculate the average time between orders per customer
order_frequency = notv_map.groupby('customer_id')['time_since_last_ads_tv'].mean().reset_index()
order_frequency.columns = ['customer_id', 'average_time_between_ads_tv']

# Merge back to the original DataFrame so each row has the avg order frequency of that customer
notv_map = notv_map.merge(order_frequency, on='customer_id', how='left')

# Display results
notv_map.head()

,customer_id,timestamp_utc,cost_milli_cent_tv,source_data,time_since_last_ads_tv,average_time_between_ads_tv_x,average_time_between_ads_tv_y
0,re006iKcKMqa0lm302,2024-03-15 01:39:37,2325.51,TV_ads,NaT,74 days 04:48:10,74 days 04:48:10
1,re006iKcKMqa0lm302,2024-05-28 06:27:47,2325.51,TV_ads,74 days 04:48:10,74 days 04:48:10,74 days 04:48:10
2,re008tXVWOF6Wfii1g,2024-04-15 22:10:21,2325.51,TV_ads,NaT,7 days 21:43:45.200000,7 days 21:43:45.200000
3,re008tXVWOF6Wfii1g,2024-04-17 23:50:19,2325.51,TV_ads,2 days 01:39:58,7 days 21:43:45.200000,7 days 21:43:45.200000
4,re008tXVWOF6Wfii1g,2024-04-19 22:26:03,2325.51,TV_ads,1 days 22:35:44,7 days 21:43:45.200000,7 days 21:43:45.200000


In [ ]:
notv_map['average_time_between_ads_tv_y'].mean()

Timedelta('6 days 06:51:51.367835793')

In [ ]:
tv_map['average_time_between_ads_tv'].mean()

Timedelta('6 days 07:31:12.492382983')

In [ ]:
tv_map['cost_milli_cent_tv'].sum()/100000

np.float64(2774.0543688000025)

In [ ]:
notv_map['cost_milli_cent_tv'].sum()/100000

np.float64(6120.719064900002)

In [ ]:
notv_map

,customer_id,timestamp_utc,cost_milli_cent_tv,source_data,time_since_last_ads_tv,average_time_between_ads_tv_x,average_time_between_ads_tv_y
0,re006iKcKMqa0lm302,2024-03-15 01:39:37,2325.51,TV_ads,NaT,74 days 04:48:10,74 days 04:48:10
1,re006iKcKMqa0lm302,2024-05-28 06:27:47,2325.51,TV_ads,74 days 04:48:10,74 days 04:48:10,74 days 04:48:10
2,re008tXVWOF6Wfii1g,2024-04-15 22:10:21,2325.51,TV_ads,NaT,7 days 21:43:45.200000,7 days 21:43:45.200000
3,re008tXVWOF6Wfii1g,2024-04-17 23:50:19,2325.51,TV_ads,2 days 01:39:58,7 days 21:43:45.200000,7 days 21:43:45.200000
4,re008tXVWOF6Wfii1g,2024-04-19 22:26:03,2325.51,TV_ads,1 days 22:35:44,7 days 21:43:45.200000,7 days 21:43:45.200000
...,...,...,...,...,...,...,...
263194,rezzpDvluO5TfkG3Dl,2024-05-25 13:17:03,2325.51,TV_ads,1 days 13:54:56,1 days 13:54:56,1 days 13:54:56
263195,rezzxE79sGAxCblaga,2024-04-30 04:20:04,2325.51,TV_ads,NaT,NaT,NaT
263196,rezzxOak8u3yiEW7xr,2024-06-26 22:59:15,2325.51,TV_ads,NaT,NaT,NaT
263197,rezzzZvkIaiWNQ1AmV,2024-06-26 23:50:31,2325.51,TV_ads,NaT,3 days 04:49:27,3 days 04:49:27


In [ ]:
tv_map

,customer_id,timestamp_utc,cost_milli_cent_tv,source_data,time_since_last_ads_tv,average_time_between_ads_tv
0,re006iKcKMqa0lm302,2024-03-15 01:39:37,2325.51,TV_ads,NaT,74 days 04:48:10
1,re006iKcKMqa0lm302,2024-05-28 06:27:47,2325.51,TV_ads,74 days 04:48:10,74 days 04:48:10
2,re008tXVWOF6Wfii1g,2024-04-15 22:10:21,2325.51,TV_ads,NaT,7 days 21:43:45.200000
3,re008tXVWOF6Wfii1g,2024-04-17 23:50:19,2325.51,TV_ads,2 days 01:39:58,7 days 21:43:45.200000
4,re008tXVWOF6Wfii1g,2024-04-19 22:26:03,2325.51,TV_ads,1 days 22:35:44,7 days 21:43:45.200000
...,...,...,...,...,...,...
119283,rezzScxmX7PK13x45S,2024-06-30 12:12:51,2325.51,TV_ads,2 days 13:24:05,2 days 13:24:05
119284,rezzZSKun4gMigGhEL,2024-03-15 21:17:45,2325.51,TV_ads,NaT,3 days 08:18:27
119285,rezzZSKun4gMigGhEL,2024-03-19 02:31:30,2325.51,TV_ads,3 days 05:13:45,3 days 08:18:27
119286,rezzZSKun4gMigGhEL,2024-03-22 22:19:55,2325.51,TV_ads,3 days 19:48:25,3 days 08:18:27


## Concat des 3 

In [ ]:
full = pd.concat([cust_after_feb, program, tv_map], ignore_index=True)

In [ ]:
full.sort_values(by=['customer_id', 'timestamp_utc'], inplace=True)

In [ ]:
full.tail(50)

,customer_id,timestamp_utc,event_name,brand,product_name,sales,quantity,source_data,campaign_name,device_type,cost_milli_cent,cost_milli_cent_tv
288217,rezztTHmLGqm4psYUY,2024-04-04 00:21:29,Order,Science Diet,SD Ca A7+ SB 15lb bg,46.99,1.0,Website_Order,NaN,NaN,NaN,NaN
1016526,rezztTHmLGqm4psYUY,2024-06-19 04:15:13,NaN,NaN,NaN,NaN,NaN,Programmatic_Ads,Contextual,Phone,336.285,NaN
1016525,rezztTHmLGqm4psYUY,2024-06-19 04:26:30,NaN,NaN,NaN,NaN,NaN,Programmatic_Ads,Contextual,Phone,242.556,NaN
288218,rezztTHmLGqm4psYUY,2024-06-19 19:19:18,Order,Science Diet,SD Ca A7+ SB 15lb bg,46.98,1.0,Website_Order,NaN,NaN,NaN,NaN
986789,rezztqaQthZfoG7kms,2024-03-05 21:50:35,NaN,NaN,NaN,NaN,NaN,Programmatic_Ads,Contextual,PC,158.398,NaN
190729,rezztqaQthZfoG7kms,2024-04-24 09:17:19,Order,Hills,HI Ca GF SBkNat Duck&Pum 12x8oz cs,7.78,1.0,Website_Order,NaN,NaN,NaN,NaN
986790,rezztqaQthZfoG7kms,2024-04-29 21:18:53,NaN,NaN,NaN,NaN,NaN,Programmatic_Ads,Retargeting,PC,2429.358,NaN
986791,rezztqaQthZfoG7kms,2024-05-27 11:56:26,NaN,NaN,NaN,NaN,NaN,Programmatic_Ads,Retargeting,PC,3805.114,NaN
862210,rezzuHEDEc0rneGJ8A,2024-06-28 22:46:14,Order,Science Diet,SD Fel Adt PerWgt Ckn 15lb bg,64.98,1.0,Website_Order,NaN,NaN,NaN,NaN
534615,rezzuPpQ1VGr7Pb0dz,2024-06-01 02:49:49,Order,Science Diet,SD Ca Adt Ckn 35lb Sh,78.98,1.0,Website_Order,NaN,NaN,NaN,NaN


In [ ]:
#full.to_csv('../data/full.csv')

In [ ]:
full.head()

,customer_id,timestamp_utc,event_name,brand,product_name,sales,quantity,source_data,campaign_name,device_type,cost_milli_cent,cost_milli_cent_tv
290002,re000fIO9QXTWYjOfn,2024-03-13 02:58:00,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,Website_Order,NaN,NaN,NaN,NaN
290003,re000fIO9QXTWYjOfn,2024-05-24 18:10:37,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,Website_Order,NaN,NaN,NaN,NaN
290004,re000fIO9QXTWYjOfn,2024-06-04 18:27:57,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,Website_Order,NaN,NaN,NaN,NaN
290005,re000fIO9QXTWYjOfn,2024-06-21 22:04:39,Order,Science Diet,SD Ca Adt SavStw S&TB Bf&Vg 12x3.5oz cs,34.32,1.0,Website_Order,NaN,NaN,NaN,NaN
443579,re000pHbVOysCXRHgt,2024-03-27 16:13:49,Order,Science Diet,SD Ca Adt Lt Ckn 30lb bg,78.99,1.0,Website_Order,NaN,NaN,NaN,NaN


In [130]:
full['campaign_name'].value_counts()

Retargeting    180150
Contextual      74329
Name: campaign_name, dtype: int64

In [ ]:
full[['cost_milli_cent', 'cost_milli_cent_tv']] = full[['cost_milli_cent', 'cost_milli_cent_tv']].fillna(0)

full['ads_tv_publisher'] = 0
full['ads_programatic'] = 0
full['ad_campaign_before_order'] = ''
full['ad_devices_before_order'] = ''
full['ad_cost_programatic'] = 0
full['ad_cost_tv_publisher'] = 0
full['ad_total_cost'] = 0



campain = []
devices = []
cost_tv = 0
cost_programatic = 0
ads_tv = 0
ads_programatic = 0

for i, row in full.iterrows():
    if row['source_data'] == 'Website_Order':
        full.at[i, 'ads_tv_publisher'] = ads_tv
        full.at[i, 'ads_programatic'] = ads_programatic
        full.at[i, 'ad_campaign_before_order'] = ', '.join(campain) if campain else 'None'
        full.at[i, 'ad_devices_before_order'] = ', '.join(devices) if devices else 'None'
        full.at[i, 'ad_cost_programatic'] = cost_programatic
        full.at[i, 'ad_cost_tv_publisher'] = cost_tv

        campain = []
        devices = []
        cost_tv = 0
        cost_programatic = 0
        cost_total = 0
        total_ads = 0
        ads_tv = 0
        ads_programatic = 0


    elif row['source_data'] in ['Programmatic_Ads', 'TV_Ads']:
        campain.append(row['campaign_name'])
        devices.append(row['device_type'])

        if row['source_data'] == 'Programmatic_Ads':
            ads_programatic += 1
        elif row['source_data'] == 'TV_Ads':
            ads_tv += 1

        try:
            cost_tv += row['cost_milli_cent_tv'].value
            cost_programatic += row['cost_milli_cent_tv'].value
        except:
            continue



# Keep only orders for analysis
full.tail(30)

,customer_id,timestamp_utc,event_name,brand,product_name,sales,quantity,source_data,campaign_name,device_type,cost_milli_cent,cost_milli_cent_tv,ads_tv_publisher,ads_programatic,ad_campaign_before_order,ad_devices_before_order,ad_cost_programatic,ad_cost_tv_publisher,ad_total_cost
683221,rezzvPZZMxU0sthAV3,2024-04-07 21:49:23,Order,Science Diet,SD Ca Adt SenSt&Sk Tk&RStew 12x12.5oz cs,46.68,1.0,Website_Order,NaN,NaN,0.000,0.0,0,0,None,None,0,0,0
683213,rezzvPZZMxU0sthAV3,2024-04-09 13:01:36,Order,Science Diet,SD Ca Adt SenSt&Sk Ckn 12x12.8oz cs,46.67,1.0,Website_Order,NaN,NaN,0.000,0.0,0,0,None,None,0,0,0
683214,rezzvPZZMxU0sthAV3,2024-04-30 11:32:13,Order,Science Diet,SD Ca Adt SenSt&Sk Ckn 12x12.8oz cs,46.67,1.0,Website_Order,NaN,NaN,0.000,0.0,0,0,None,None,0,0,0
683215,rezzvPZZMxU0sthAV3,2024-05-06 22:52:36,Order,Science Diet,SD Ca Adt SenSt&Sk Ckn 12x12.8oz cs,93.34,2.0,Website_Order,NaN,NaN,0.000,0.0,0,0,None,None,0,0,0
683216,rezzvPZZMxU0sthAV3,2024-05-17 23:05:28,Order,Science Diet,SD Ca Adt SenSt&Sk Ckn 12x12.8oz cs,93.34,2.0,Website_Order,NaN,NaN,0.000,0.0,0,0,None,None,0,0,0
683217,rezzvPZZMxU0sthAV3,2024-05-30 11:25:40,Order,Science Diet,SD Ca Adt SenSt&Sk Ckn 12x12.8oz cs,93.34,2.0,Website_Order,NaN,NaN,0.000,0.0,0,0,None,None,0,0,0
683218,rezzvPZZMxU0sthAV3,2024-06-08 22:48:51,Order,Science Diet,SD Ca Adt SenSt&Sk Ckn 12x12.8oz cs,93.34,2.0,Website_Order,NaN,NaN,0.000,0.0,0,0,None,None,0,0,0
683219,rezzvPZZMxU0sthAV3,2024-06-22 00:02:03,Order,Science Diet,SD Ca Adt SenSt&Sk Ckn 12x12.8oz cs,46.67,1.0,Website_Order,NaN,NaN,0.000,0.0,0,0,None,None,0,0,0
683220,rezzvPZZMxU0sthAV3,2024-06-30 20:15:27,Order,Science Diet,SD Ca Adt SenSt&Sk Ckn 12x12.8oz cs,93.34,2.0,Website_Order,NaN,NaN,0.000,0.0,0,0,None,None,0,0,0
549381,rezzvb0EMKNiWWP7ky,2024-04-05 02:24:47,Order,Science Diet,SD Ca Adt Ckn&BlyEnt 12x13oz cs,41.88,1.0,Website_Order,NaN,NaN,0.000,0.0,0,0,None,None,0,0,0


In [132]:
chrono = pd.read_csv('../data/chrono_with_float.csv')

In [134]:
chrono.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3364592 entries, 0 to 3364591
Data columns (total 17 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Unnamed: 0.1     int64  
 1   Unnamed: 0       int64  
 2   customer_id      object 
 3   timestamp_utc    object 
 4   event_name       object 
 5   brand            object 
 6   product_name     object 
 7   sales            float64
 8   quantity         float64
 9   kind             object 
 10  dsp_id           object 
 11  device_id        object 
 12  campaign_name    object 
 13  device_type      object 
 14  cost_milli_cent  float64
 15  chrono           object 
 16  chrono_float     float64
dtypes: float64(4), int64(2), object(11)
memory usage: 436.4+ MB


In [136]:
chrono = chrono.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'chrono', 'chrono_float'])

In [137]:
chrono = chrono.rename({'cost_milli_cent':'cost_milli_cent_programatic'})

In [140]:
chrono_mini = chrono[['customer_id', 'timestamp_utc']].drop_duplicates()

In [141]:
tv

,device_id,timestamp_utc,cost_milli_cent
0,ctv81YlbBXho,2024-04-23 21:09:46,2325.51
1,ctvWr7bOO5Je,2024-04-19 18:31:30,2325.51
2,ctvktBqDUgcV,2024-05-07 23:32:37,2325.51
3,ctv4bFFpBJms,2024-06-28 00:10:10,2325.51
4,ctvFZB9pUMZ7,2024-05-24 03:45:45,2325.51
...,...,...,...
5827128,ctvyq3Mv8Wyi,2024-05-07 02:27:30,2325.51
5827129,ctvyq3Mv8Wyi,2024-05-08 02:06:55,2325.51
5827130,ctvAxmMKz1Po,2024-05-02 02:28:27,2325.51
5827131,ctv4IsWGzGJ9,2024-03-13 03:43:02,2325.51


In [ ]:
tv = chrono_mini.merge(tv)

In [133]:
chrono.head(20)

,Unnamed: 0.1,Unnamed: 0,customer_id,timestamp_utc,event_name,brand,product_name,sales,quantity,kind,dsp_id,device_id,campaign_name,device_type,cost_milli_cent,chrono,chrono_float
0,0,8993084,re0007V8sqIHsZnbvC,2024-01-29 23:46:22,Product Page View,Science Diet,SD Fel Adt PerWgt Ckn 15lb bg,NaN,NaN,website,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
1,1,8993083,re0007V8sqIHsZnbvC,2024-01-29 23:54:51,Product Page View,Science Diet,SD Fel Adt PerWgt Ckn 15lb bg,NaN,NaN,website,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
2,2,6716743,re000JYhnKbTkPqMB4,2024-01-08 06:10:19,Add to cart,Science Diet,SD Ca Adt PerWgt Ckn 25lb bg,NaN,NaN,website,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
3,3,6716744,re000JYhnKbTkPqMB4,2024-01-08 06:31:49,Product Page View,Science Diet,SD Ca Adt PerWgt Ckn 4lb bg,NaN,NaN,website,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
4,4,6716742,re000JYhnKbTkPqMB4,2024-01-08 06:39:11,Product Page View,Science Diet,SD Ca Adt PerWgt Ckn 25lb bg,NaN,NaN,website,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
5,5,10702452,re000fIO9QXTWYjOfn,2024-01-24 14:25:56,NaN,NaN,NaN,NaN,NaN,ads,dspIYJLpU5Ex3KiyA1K3wn7VNexZWB,ctvQEmtlTLvO,Retargeting,TV,3595.156,0 days 00:00:00,0.00000
6,6,3405010,re000fIO9QXTWYjOfn,2024-01-24 14:44:47,Add to cart,Science Diet,SD Ca A7+ SavStw S&TB Ckn&Vg 12x3.5oz cs,NaN,NaN,website,NaN,NaN,NaN,NaN,NaN,0 days 00:18:51,0.01309
7,7,3405006,re000fIO9QXTWYjOfn,2024-01-24 14:48:51,Product Page View,Science Diet,SD Ca A7+ SavStw S&TB Ckn&Vg 12x3.5oz cs,NaN,NaN,website,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
8,8,3405008,re000fIO9QXTWYjOfn,2024-01-24 14:51:54,Order,Science Diet,SD Ca A7+ SavStw S&TB Ckn&Vg 12x3.5oz cs,31.92,1.0,website,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
9,9,3405005,re000fIO9QXTWYjOfn,2024-01-24 14:55:03,Product Page View,Science Diet,SD Ca A7+ SavStw S&TB Ckn&Vg 12x3.5oz cs,NaN,NaN,website,NaN,NaN,NaN,NaN,NaN,NaN,0.00000
